In [2]:
import json
import xml.etree.ElementTree as ET
from collections import defaultdict
from pathlib import Path, PureWindowsPath, PurePath

In [4]:
def addAnnoItem(image_id, category_id, bbox, annotation_id):
    annotation_item = dict()
    annotation_item["segmentation"] = []
    seg = []
    # bbox[] is x,y,w,h
    # left_bottom
    seg.append(bbox[0])
    seg.append(bbox[1])
    # left_top
    seg.append(bbox[0])
    seg.append(bbox[1] + bbox[3])
    # right_top
    seg.append(bbox[0] + bbox[2])
    seg.append(bbox[1] + bbox[3])
    # right_bottom
    seg.append(bbox[0] + bbox[2])
    seg.append(bbox[1])

    annotation_item["segmentation"].append(seg)

    annotation_item["area"] = bbox[2] * bbox[3]
    annotation_item["iscrowd"] = 0
    annotation_item["ignore"] = 0
    annotation_item["image_id"] = image_id
    annotation_item["bbox"] = bbox
    annotation_item["category_id"] = category_id
    annotation_id += 1
    annotation_item["id"] = annotation_id

    return annotation_item


def addCtgItem(category_id, object_name):
    categories = dict()
    categories["supercategory"] = "none"
    categories_id = category_id
    categories["id"] = categories_id
    categories["name"] = object_name
    return categories


def parseXmlFiles(
    xml_path: str,
    json_save_path: str,
    coco_type: str = "instances",
    ctg_id_start: int = 1,
    categories_set: dict = {},
    force_category = False,
):
    coco = defaultdict(list)
    coco["images"], coco["categories"], coco["annotations"] = [], [], []
    coco["type"] = coco_type
    category_set = categories_set.copy()

    if len(category_set) != 0:
        for key, item in category_set.items():
            categories = addCtgItem(item, key)
            coco["categories"].append(categories)

    path = Path(xml_path)

    for f in path.iterdir():
        if f.suffix != ".xml":
            continue

        tree = ET.parse(f)
        root = tree.getroot()
        if root.tag != "annotation":
            raise Exception(
                "pascal voc xml root element should be annotation, rather than {}".format(
                    root.tag
                )
            )
        image = dict()
        # elem is <folder>, <filename>, <size>, <object>
        for elem in root:
            if elem.tag == "folder":
                continue

            if elem.tag == "filename":

                file_name = elem.text
                image["file_name"] = file_name
                image["id"] = len(coco["images"])

            if elem.tag == "size":
                for subelem in elem:
                    if subelem.tag == "width" or subelem.tag == "height":
                        image[subelem.tag] = int(subelem.text)

            if len(image) == 4 and len(coco["images"]) == image["id"]:
                coco["images"].append(image)
                print("add image with {}".format(file_name))

            if elem.tag == "object":
                bbox = dict()
                # categories_id = None
                for subelem in elem:
                    if subelem.tag == "name":
                        object_name = subelem.text
                        if object_name not in category_set:
                            categories = addCtgItem(
                                len(category_set) + ctg_id_start, object_name
                            )
                            categories_id = len(category_set) + ctg_id_start
                            category_set[object_name] = categories_id
                            if not force_category:
                                coco["categories"].append(categories)
                            # print(category_set)
                        else:
                            categories_id = category_set[object_name]
                    if subelem.tag == "bndbox":
                        for e in subelem:
                            bbox[e.tag] = int(e.text)

                    if len(bbox) > 0:
                        bndbox = []
                        # x
                        bndbox.append(bbox["xmin"])
                        # y
                        bndbox.append(bbox["ymin"])
                        # w
                        bndbox.append(bbox["xmax"] - bbox["xmin"])
                        # h
                        bndbox.append(bbox["ymax"] - bbox["ymin"])

                        # print(categories, image)
                        if force_category:
                            if categories_id in categories_set.values():
                                coco["annotations"].append(
                                    addAnnoItem(
                                        image["id"],
                                        categories_id,
                                        bndbox,
                                        len(coco["annotations"]),
                                    )
                                )
                        else:
                            coco["annotations"].append(
                                addAnnoItem(
                                    image["id"],
                                    categories_id,
                                    bndbox,
                                    len(coco["annotations"]),
                                )
                            )

    json.dump(coco, open(json_save_path, "w"))
    print("Total categories:{}".format(list(category_set.keys())))
    if force_category:
        print("forced categories:{}".format(list(categories_set.keys())))


In [7]:
path = Path.cwd()

ann_path_train = path / 'datasets/untunnel-detection/annotations_xml/train'
ann_path_val =  path / 'datasets/untunnel-detection/annotations_xml/val'
img_path_train = path / 'datasets/untunnel-detection/train'
img_path_val =  path / 'datasets/untunnel-detection/val'

json_save_path = path / 'datasets/untunnel-detection/annotations/instances'

In [8]:
obj_list=[
    "spiledmaterial",
    "person",
]

In [10]:
ctg_dict = dict()
_ind = 1
for i in obj_list:
    ctg_dict[i] = _ind
    _ind += 1
print(ctg_dict)

{'spiledmaterial': 1, 'person': 2}


In [11]:
parseXmlFiles(ann_path_train,str(json_save_path)+'_test.json',categories_set=ctg_dict,force_category=True)

add image with JS275.jpg
add image with DLLJ729.jpg
add image with LF384.jpg
add image with KC3489.jpg
add image with 334.jpg
add image with train5716.jpg
add image with train3886.jpg
add image with KC240.jpg
add image with LF339.jpg
add image with KC799.jpg
add image with 1497.jpg
add image with JS5.jpg
add image with DLLJ454.jpg
add image with 899.jpg
add image with ZW2246.jpg
add image with ZW2.jpg
add image with DLLJ530.jpg
add image with 482.jpg
add image with 373.jpg
add image with 399.jpg
add image with KC3530.jpg
add image with LF44.jpg
add image with DLLJ306.jpg
add image with DLLJ396.jpg
add image with train4139.jpg
add image with 4675.jpg
add image with DLLJ311.jpg
add image with 324.jpg
add image with 161.jpg
add image with ZW427.jpg
add image with DLLJ143.jpg
add image with 114.jpg
add image with KC6369.jpg
add image with ZW442.jpg
add image with JS13280.jpg
add image with DLLJ797.jpg
add image with 4678.jpg
add image with 4780.jpg
add image with DLLJ723.jpg
add image with